# Giới thiệu 
__Hệ thống CBIR (truy xuất hình ảnh dựa trên nội dung)__

__Trích xuất đặc trưng của hình ảnh truy vấn và truy xuất các đặc trưng tương tự từ cơ sở dữ liệu hình ảnh__

<img align='center' style="border-color:gray;border-width:2px;border-style:dashed"   src='CBIR.png' padding='5px' height="300px"></img>

# Trích rút đặc trưng:
Trong phần này chúng ta thực hiện các phương pháp trích rút đặc trưng phổ biến:
- Shape-based
    +  Egde Histogram
    +  HOG

### Đọc dữ liệu và tạo ra file data.csv
Xóa file data.csv trước khi chạy

In [2]:
db = Database()
data = db.get_data()

# shape-based
 - [edge histogram]
 - [HOG (histogram of gradient)]

# Edge histogram

In [1]:
from __future__ import print_function

from src.evaluate import evaluate_class
from src.DB import Database

from six.moves import cPickle
import numpy as np
import scipy.misc
from math import sqrt
import os

In [2]:
stride = (1, 1)
n_slice  = 10
h_type   = 'region'
d_type   = 'cosine'

depth    = 5

''' MMAP    
      depth
       depthNone, region-stride(1, 1)-n_slice10,co, MMAP 0.101670982288
       depth100,  region-stride(1, 1)-n_slice10,co, MMAP 0.207817305128
       depth30,   region-stride(1, 1)-n_slice10,co, MMAP 0.291715090839
       depth10,   region-stride(1, 1)-n_slice10,co, MMAP 0.353722379063
       depth5,    region-stride(1, 1)-n_slice10,co, MMAP 0.367119444444
       depth3,    region-stride(1, 1)-n_slice10,co, MMAP 0.3585
       depth1,    region-stride(1, 1)-n_slice10,co, MMAP 0.302
  
       (exps below use depth=None)
  
      d_type
       global-stride(2, 2),d1, MMAP 0.0530993236031
       global-stride(2, 2),co, MMAP 0.0528310744618
  
      stride
       region-stride(2, 2)-n_slice4,d1, MMAP 0.0736245142237
       region-stride(1, 1)-n_slice4,d1, MMAP 0.0704206226545
  
      n_slice
       region-stride(1, 1)-n_slice10,co, MMAP 0.101670982288
       region-stride(1, 1)-n_slice6,co, MMAP 0.0977736743859
  
      h_type
       global-stride(2, 2),d1, MMAP 0.0530993236031
       region-stride(2, 2)-n_slice4,d1, MMAP 0.0736245142237
'''

edge_kernels = np.array([
  [
   # vertical
   [1,-1], 
   [1,-1]
  ],
  [
   # horizontal
   [1,1], 
   [-1,-1]
  ],
  [
   # 45 diagonal
   [sqrt(2),0], 
   [0,-sqrt(2)]
  ],
  [
   # 135 diagnol
   [0,sqrt(2)], 
   [-sqrt(2),0]
  ],
  [
   # non-directional
#################Your code here (1)######################
   [None,None], 
   [None,None]
#####################################################
  ]
])

# cache dir
cache_dir = 'cache'
if not os.path.exists(cache_dir):
  os.makedirs(cache_dir)

In [3]:
class Edge(object):

  def histogram(self, input, stride=(2, 2), type=h_type, n_slice=n_slice, normalize=True):
    ''' count img histogram
  
      arguments
        input    : a path to a image or a numpy.ndarray
        stride   : stride of edge kernel
        type     : 'global' means count the histogram for whole image
                   'region' means count the histogram for regions in images, then concatanate all of them
        n_slice  : work when type equals to 'region', height & width will equally sliced into N slices
        normalize: normalize output histogram
  
      return
        type == 'global'
          a numpy array with size len(edge_kernels)
        type == 'region'
          a numpy array with size len(edge_kernels) * n_slice * n_slice
    '''
    if isinstance(input, np.ndarray):  # examinate input type
      img = input.copy()
    else:
      img = scipy.misc.imread(input, mode='RGB')
    ############Your code here (2)###############
    height, width, channel = None
    ##########################################
    if type == 'global':
      hist = self._conv(img, stride=stride, kernels=edge_kernels)
  
    elif type == 'region':
      hist = np.zeros((n_slice, n_slice, edge_kernels.shape[0]))
      h_silce = np.around(np.linspace(0, height, n_slice+1, endpoint=True)).astype(int)
        ############Your code here (3)#######################
      w_slice = None
        ##################################################
  
      for hs in range(len(h_silce)-1):
        for ws in range(len(w_slice)-1):
          img_r = img[h_silce[hs]:h_silce[hs+1], w_slice[ws]:w_slice[ws+1]]  # slice img to regions
          hist[hs][ws] = self._conv(img_r, stride=stride, kernels=edge_kernels)
  
    if normalize:
      hist /= np.sum(hist)
  
    return hist.flatten()
  
  
  def _conv(self, img, stride, kernels, normalize=True):
    H, W, C = img.shape
    conv_kernels = np.expand_dims(kernels, axis=3)
    conv_kernels = np.tile(conv_kernels, (1, 1, 1, C))
    assert list(conv_kernels.shape) == list(kernels.shape) + [C]  # check kernels size
  
    sh, sw = stride
    kn, kh, kw, kc = conv_kernels.shape
  
    hh = int((H - kh) / sh + 1)
    
    ###############Your code here (4)#############
    ww = None
    
    ##############################################
  
    hist = np.zeros(kn)
  
    for idx, k in enumerate(conv_kernels):
      for h in range(hh):
        hs = int(h*sh)
        he = int(h*sh + kh)
        for w in range(ww):
          ws = w*sw
          we = w*sw + kw
          hist[idx] += np.sum(img[hs:he, ws:we] * k)  # element-wise product
  
    if normalize:
      hist /= np.sum(hist)
  
    return hist
  
  
  def make_samples(self, db, verbose=True):
    if h_type == 'global':
      sample_cache = "edge-{}-stride{}".format(h_type, stride)
    elif h_type == 'region':
      sample_cache = "edge-{}-stride{}-n_slice{}".format(h_type, stride, n_slice)
  
    try:
      samples = cPickle.load(open(os.path.join(cache_dir, sample_cache), "rb", True))
      for sample in samples:
        sample['hist'] /= np.sum(sample['hist'])  # normalize
      if verbose:
        print("Using cache..., config=%s, distance=%s, depth=%s" % (sample_cache, d_type, depth))
    except:
      if verbose:
        print("Counting histogram..., config=%s, distance=%s, depth=%s" % (sample_cache, d_type, depth))
  
      samples = []
      data = db.get_data()
      for d in data.itertuples():
        d_img, d_cls = getattr(d, "img"), getattr(d, "cls")
        ###############Your code here(5)#################
        d_hist = None # histogram
        ##########################################################
        samples.append({
                        'img':  d_img, 
                        'cls':  d_cls, 
                        'hist': d_hist
                      })
      cPickle.dump(samples, open(os.path.join(cache_dir, sample_cache), "wb", True))
  
    return samples

In [4]:
db = Database()

# check shape
assert edge_kernels.shape == (5, 2, 2)

  # evaluate database
APs = evaluate_class(db, f_class=Edge, d_type=d_type, depth=depth)
cls_MAPs = []
for cls, cls_APs in APs.items():
    MAP = np.mean(cls_APs)
    print("Class {}, MAP {}".format(cls, MAP))
    cls_MAPs.append(MAP)
print("MMAP", np.mean(cls_MAPs))

Using cache..., config=edge-region-stride(1, 1)-n_slice10, distance=cosine, depth=5
Class database\obj_bus, MAP 0.0
Class database\eat_feasts, MAP 0.019833333333333335
Class database\obj_dish, MAP 0.0
Class database\art_cybr, MAP 0.11133333333333333
Class database\obj_decoys, MAP 0.0
Class database\bld_lighthse, MAP 0.07994444444444444
Class database\bld_modern, MAP 0.1326449275362319
Class database\art_mural, MAP 0.0
Class database\obj_aviation, MAP 0.0
Class database\art_1, MAP 0.8448750000000002
Class database\fitness, MAP 0.0
Class database\obj_car, MAP 0.0
MMAP 0.09905258655394526


# HOG (histogram of gradient)

In [12]:
from __future__ import print_function

from src.evaluate import evaluate_class
from src.DB import Database

from skimage.feature import hog
from skimage import color

from six.moves import cPickle
import numpy as np
import scipy.misc
import os

In [13]:
n_bin    = 10
n_slice  = 6
n_orient = 8
p_p_c    = (2, 2)
c_p_b    = (1, 1)
h_type   = 'region'
d_type   = 'd1'

depth    = 5

''' MMAP
     depth
      depthNone, HOG-region-n_bin10-n_slice6-n_orient8-ppc(2, 2)-cpb(1, 1), distance=d1, MMAP 0.155887235348
      depth100,  HOG-region-n_bin10-n_slice6-n_orient8-ppc(2, 2)-cpb(1, 1), distance=d1, MMAP 0.261149622088
      depth30,   HOG-region-n_bin10-n_slice6-n_orient8-ppc(2, 2)-cpb(1, 1), distance=d1, MMAP 0.371054105819
      depth10,   HOG-region-n_bin10-n_slice6-n_orient8-ppc(2, 2)-cpb(1, 1), distance=d1, MMAP 0.449627835097
      depth5,    HOG-region-n_bin10-n_slice6-n_orient8-ppc(2, 2)-cpb(1, 1), distance=d1, MMAP 0.465333333333
      depth3,    HOG-region-n_bin10-n_slice6-n_orient8-ppc(2, 2)-cpb(1, 1), distance=d1, MMAP 0.463833333333
      depth1,    HOG-region-n_bin10-n_slice6-n_orient8-ppc(2, 2)-cpb(1, 1), distance=d1, MMAP 0.398

      (exps below use depth=None)

     ppc & cpb
      HOG-global-n_bin10-n_orient8-ppc(2, 2)-cpb(1, 1), distance=d1, MMAP 0.105569494513
      HOG-global-n_bin10-n_orient8-ppc(32, 32)-cpb(1, 1), distance=d1, MMAP 0.0945343258574
      HOG-global-n_bin10-n_orient8-ppc(8, 8)-cpb(3, 3), distance=d1, MMAP 0.0782408187317

     h_type
      HOG-global-n_bin100-n_orient8-ppc(32, 32)-cpb(1, 1), distance=d1, MMAP 0.0990826443803
      HOG-region-n_bin100-n_slice4-n_orient8-ppc(32, 32)-cpb(1, 1), distance=d1, MMAP 0.131164310773

     n_orient
      HOG-global-n_bin10-n_orient8-ppc(2, 2)-cpb(1, 1), distance=d1, MMAP 0.105569494513
      HOG-region-n_bin10-n_slice4-n_orient18-ppc(2, 2)-cpb(1, 1), distance=d1, MMAP 0.14941454752

     n_bin
      HOG-region-n_bin5-n_slice4-n_orient8-ppc(32, 32)-cpb(1, 1), distance=d1, MMAP 0.140448910465
      HOG-region-n_bin10-n_slice4-n_orient8-ppc(32, 32)-cpb(1, 1), distance=d1, MMAP 0.144675311048
      HOG-region-n_bin20-n_slice4-n_orient8-ppc(32, 32)-cpb(1, 1), distance=d1, MMAP 0.1429074023
      HOG-region-n_bin100-n_slice4-n_orient8-ppc(32, 32)-cpb(1, 1), distance=d1, MMAP 0.131164310773

     n_slice
      HOG-region-n_bin10-n_slice2-n_orient8-ppc(2, 2)-cpb(1, 1), distance=d1, MMAP 0.116513458785
      HOG-region-n_bin10-n_slice4-n_orient8-ppc(2, 2)-cpb(1, 1), distance=d1, MMAP 0.151557545391
      HOG-region-n_bin10-n_slice6-n_orient8-ppc(2, 2)-cpb(1, 1), distance=d1, MMAP 0.155887235348
      HOG-region-n_bin10-n_slice8-n_orient8-ppc(2, 2)-cpb(1, 1), distance=d1, MMAP 0.15347983005
'''

# cache dir
cache_dir = 'cache'
if not os.path.exists(cache_dir):
  os.makedirs(cache_dir)

In [14]:
class HOG(object):

  def histogram(self, input, n_bin=n_bin, type=h_type, n_slice=n_slice, normalize=True):
    ''' count img histogram
  
      arguments
        input    : a path to a image or a numpy.ndarray
        n_bin    : number of bins of histogram
        type     : 'global' means count the histogram for whole image
                   'region' means count the histogram for regions in images, then concatanate all of them
        n_slice  : work when type equals to 'region', height & width will equally sliced into N slices
        normalize: normalize output histogram
  
      return
        type == 'global'
          a numpy array with size n_bin
        type == 'region'
          a numpy array with size n_bin * n_slice * n_slice
    '''
    if isinstance(input, np.ndarray):  # examinate input type
      img = input.copy()
    else:
      img = scipy.misc.imread(input, mode='RGB')
    height, width, channel = img.shape
  
    if type == 'global':
      hist = self._HOG(img, n_bin)
  
    elif type == 'region':
      hist = np.zeros((n_slice, n_slice, n_bin))
        #######################Your code here (6)#########################
      h_silce = None
      w_slice = None
        ##############################################################
  
      for hs in range(len(h_silce)-1):
        for ws in range(len(w_slice)-1):
            #############Your code here (7)#############
          img_r = None
          hist[hs][ws] = None
            ########################################
  
    if normalize:
      hist /= np.sum(hist)
  
    return hist.flatten()

  def _HOG(self, img, n_bin, normalize=True):
    image = color.rgb2gray(img)
    fd = hog(image, orientations=n_orient, pixels_per_cell=p_p_c, cells_per_block=c_p_b)
    bins = np.linspace(0, np.max(fd), n_bin+1, endpoint=True)
    hist, _ = np.histogram(fd, bins=bins)
  
    if normalize:
      hist = np.array(hist) / np.sum(hist)
  
    return hist

  def make_samples(self, db, verbose=True):
    if h_type == 'global':
      sample_cache = "HOG-{}-n_bin{}-n_orient{}-ppc{}-cpb{}".format(h_type, n_bin, n_orient, p_p_c, c_p_b)
    elif h_type == 'region':
      sample_cache = "HOG-{}-n_bin{}-n_slice{}-n_orient{}-ppc{}-cpb{}".format(h_type, n_bin, n_slice, n_orient, p_p_c, c_p_b)
  
    try:
      samples = cPickle.load(open(os.path.join(cache_dir, sample_cache), "rb", True))
      for sample in samples:
        sample['hist'] /= np.sum(sample['hist'])  # normalize
      if verbose:
        print("Using cache..., config=%s, distance=%s, depth=%s" % (sample_cache, d_type, depth))
    except:
      if verbose:
        print("Counting histogram..., config=%s, distance=%s, depth=%s" % (sample_cache, d_type, depth))

      samples = []
      data = db.get_data()
      for d in data.itertuples():
        d_img, d_cls = getattr(d, "img"), getattr(d, "cls")
        d_hist = self.histogram(d_img, type=h_type, n_slice=n_slice)
        samples.append({
                        'img':  d_img, 
                        'cls':  d_cls, 
                        'hist': d_hist
                      })
      cPickle.dump(samples, open(os.path.join(cache_dir, sample_cache), "wb", True))

    return samples

In [15]:
db = Database()

# evaluate database
###########Your code here (8) ####################
APs = None
###########################################
cls_MAPs = []
for cls, cls_APs in APs.items():
    MAP = np.mean(cls_APs)
    print("Class {}, MAP {}".format(cls, MAP))
    cls_MAPs.append(MAP)
print("MMAP", np.mean(cls_MAPs))

Counting histogram..., config=HOG-region-n_bin10-n_slice6-n_orient8-ppc(2, 2)-cpb(1, 1), distance=d1, depth=5


C:\Users\admin\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:23: DeprecationWarning:     `imread` is deprecated!
    `imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``imageio.imread`` instead.


Class database\obj_dish, MAP 0.9395555555555556
Class database\art_mural, MAP 0.35883333333333334
Class database\obj_aviation, MAP 0.25727777777777777
Class database\obj_bus, MAP 0.6092638888888889
Class database\fitness, MAP 0.9689444444444445
Class database\art_cybr, MAP 0.658111111111111
Class database\bld_lighthse, MAP 0.30852777777777773
Class database\bld_modern, MAP 0.4432669082125604
Class database\obj_car, MAP 0.5708638211382114
Class database\art_1, MAP 0.228
Class database\eat_feasts, MAP 0.16433333333333333
Class database\obj_decoys, MAP 0.9625833333333333
MMAP 0.5391301070755272
